In [61]:
import gc
import numpy as np
import time
import pytz
import datetime
from tqdm import tqdm
import logging
import traceback
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os

In [62]:
df = pd.read_excel('/content/DomainExtract.xlsx')

In [63]:
df['Postcode'] = df['SUBURB'].apply(lambda x: x[-4:].title())
df['State'] = df['SUBURB'].apply(lambda x: x.split('-')[-2].title())
df['Suburb Name'] = df['SUBURB'].apply(lambda x: ' '.join(x.split('-')[:-2]).title())

In [64]:
del df['Date']

In [65]:
state_names = {
    'Tas': 'Tasmania',
    'Nsw': 'New South Wales',
    'Qld': 'Queensland',
    'Nt': 'Northern Territory',
    'Vic': 'Victoria',
    'Sa': 'South Australia',
    'Act': 'Australian Capital Territory',
    'Wa': 'Western Australia'
    }
df['State'] = df['State'].replace(state_names)

In [66]:
new_column_names = {col: col.title() for col in df.columns}
df = df.rename(columns=new_column_names)
df.head()

,Bedrooms,Type,Median Price,Avg Days On Market,Clearance Rate,Sold This Year,Population,Average Age,Owner,Renter,Family,Single,Suburb,Postcode,State,Suburb Name
0,2,House,-,-,-,1,3103,20 to 39,0.63,0.37,0.48,0.52,trinity-park-qld-4879,4879,Queensland,Trinity Park
1,3,House,$570k,51 days,-,16,3103,20 to 39,0.63,0.37,0.48,0.52,trinity-park-qld-4879,4879,Queensland,Trinity Park
2,4,House,$635k,37 days,-,70,3103,20 to 39,0.63,0.37,0.48,0.52,trinity-park-qld-4879,4879,Queensland,Trinity Park
3,5,House,$735k,-,-,11,3103,20 to 39,0.63,0.37,0.48,0.52,trinity-park-qld-4879,4879,Queensland,Trinity Park
4,2,House,$340k,112 days,-,22,9701,60+,0.68,0.32,0.44,0.56,portland-vic-3305,3305,Victoria,Portland


In [67]:
custom_order = ['Postcode', 'State', 'Suburb Name','Bedrooms', 'Type', 'Median Price', 'Avg Days On Market',
       'Clearance Rate', 'Sold This Year', 'Population', 'Average Age',
       'Owner', 'Renter', 'Family', 'Single', 'Suburb']  # Define your custom order of column names
df = df.reindex(columns=custom_order)

In [68]:
df.head()

,Postcode,State,Suburb Name,Bedrooms,Type,Median Price,Avg Days On Market,Clearance Rate,Sold This Year,Population,Average Age,Owner,Renter,Family,Single,Suburb
0,4879,Queensland,Trinity Park,2,House,-,-,-,1,3103,20 to 39,0.63,0.37,0.48,0.52,trinity-park-qld-4879
1,4879,Queensland,Trinity Park,3,House,$570k,51 days,-,16,3103,20 to 39,0.63,0.37,0.48,0.52,trinity-park-qld-4879
2,4879,Queensland,Trinity Park,4,House,$635k,37 days,-,70,3103,20 to 39,0.63,0.37,0.48,0.52,trinity-park-qld-4879
3,4879,Queensland,Trinity Park,5,House,$735k,-,-,11,3103,20 to 39,0.63,0.37,0.48,0.52,trinity-park-qld-4879
4,3305,Victoria,Portland,2,House,$340k,112 days,-,22,9701,60+,0.68,0.32,0.44,0.56,portland-vic-3305


In [69]:
df.columns

Index(['Postcode', 'State', 'Suburb Name', 'Bedrooms', 'Type', 'Median Price',
       'Avg Days On Market', 'Clearance Rate', 'Sold This Year', 'Population',
       'Average Age', 'Owner', 'Renter', 'Family', 'Single', 'Suburb'],
      dtype='object')

In [70]:
df = df.sort_values(by=['Postcode', 'Suburb Name'])
df = df.reset_index(drop=True)
df.index += 1

In [79]:
df['Avg Days On Market'] = df['Avg Days On Market'].str.extract('(\d+)').replace(np.nan, '0').astype(int)

In [81]:
df.to_excel('DomainExtract.xlsx',index=False)